# **Semesteroppgave**

I denne oppgaven har jeg valgt å gå gjennom et annuitetslån. Her ser jeg på hvilke utfordringer en vanlig familie på fire, med medianinntekt i Norge møter på.

Jeg har valgt å behandle denne oppgaven fordi jeg ønsker bedre forsåelse i hvordan et annuitetslån er bygd opp. En annuitetslånkalkulator kan jeg selv benytte om jeg kommer i en situasjon der jeg skal søke om et lån til for eksempel en bolig. I tillegg gjør problemstilleng min det at jeg har mulighet til å få innsikt i hvilke utfordrigner medianfamilier i Norge møter på økonomisk i dag.

Jeg har ikke gjennomført en lignende oppgave før, og alt innhold er laget av meg. Av litteratur og kilder har jeg hentet data på medianinntekt fra SSB "https://www.ssb.no/97014/median-inntekt-etter-skatt-etter-fylke-og-husholdningstype.kroner". Her har jeg valgt ut medianinntekten til et par med to barn der yngste barnet er mellom null og fem år. Jeg har også brukt SIFOs kalkulator "https://www.oslomet.no/om/sifo/referansebudsjettet" for å se på de månedlige utgiftene til denne familien. I tilegg har jeg sett på de effektive rentene hos bankene Sparebank 1, DNB, Nordea og Storebrand.

For å få en bedre forståelse av hvordan en skal gå frem på en slik oppgave startet jeg arbeidet med å søke opp eksempler på en annuitetslån kalkulator. Det eksisterer mange gode Python-scripts på blant annet Github, og mange fine videoer på YouTube. En av de første videoene jeg møtte på lignet på noe så enkelt som det her:

In [2]:
def kalkulator(lån, rente, terminer):
    a = (lån * rente*((1+rente)**terminer))
    b = (((1+rente)**terminer)-1)
    c = (a/b)
    return c

In [3]:
kalkulator(1000000, 0.02, 10)

111326.52786531636

Men jeg forsto gange tidlig at dette ikke var nok til en semester oppgave. Så til oppgave løsingen startet jeg med det å lage selve input delen, der brukeren av kalkulatoren fyller inn størrelsen på lånet, renten og antall terminer. Deretter lagde jeg utregningsdelen, der terminene ganges med 12 for å få antall måneder, renten endres fra heltall til desimaltall (integer til string) og terminbeløpet regnes ut.Deretter lagde jeg oversiktstabellen, som gjør utregninger ved bruk av terminbeløpet for å finne ut hvor stor del som er rente, hvor stor del som er avdrag og hva som gjenstår av restlån. I tillegg formaterte jeg dette inn i en tabell slik at den skulle være oversiktlig nok. Til slutt lagde jeg en liten bit som viste grafisk hvordan rente andelen avtar og avdragene øker over tid, samt også det som får kalkulatoren til å fungere.

Familien:
Familien Hanssen består av fire familiemedlemmer, pappa Jørn, mamma Trine og de to barna deres Lars (5) og Lise (2)

De har til nå bodd i et rekkehus som de har leid av et utleier, men ønsker å kjøpe et hus selv. De har en samlet årlig inntekt på 648 000 kr etter skatt og har ingen gjeld fra før. De har i tillegg klart å spare 500 000 kr i egenkapital. 

Ifølge referansebudsjettet til Oslomet vil familien ha et månedlig forbruk på 33 229 kr, mens deres månedlig inntekt etter skatt (648 000 / 12) er 54 000. Det betyr at de har mulighet til å betale ned et lån som maksimalt har terminbeløp på 20 771 kr.

De har en ide om hvilke prisklasse de ligger i når det kommer til huskjøp, omtrent 6 000 000 kr, og formidler seg i banken om lånemuligheter.

Hos Sparebank 1 får de tilbud om et lån på 5 500 000 kr, en rente på 2,14% og en nedbetalingstid på 30 år. Dette ønsker de å undersøke nermere, for å se hvor mye de må betale både årlig og månedlig. 

Familien bestemmer seg for å oppsøke flere banker for å se om de kan få en bedre låneavtale. I DNB får de tilbud om et lån av samme størrelse med lavere rente, 2,02% men med kortere nedbetalingstid, 27 år. De vil derfor sammenligne de to tilbude for å se hvilket som passer de best.

In [7]:
import array
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


def lånInfo():
    lånmen = int(input("\nLån størrelse:\n"))
    rente = float(input("\nRente i prosent:\n"))
    terminer = int(input("\nAntall terminer:\n"))
    while terminer >= 31 or terminer <= 0:
        terminer = int(input("\nBenytt antall terminer mellom 1 og 30\n"))
    return lånmen, rente, terminer


def beregneMånedligeBetalinger(lånmen, rente, terminer):
    måneder = terminer * 12
    rentePros = (rente * 0.1) / måneder
    terminbeløp = (lånmen * rentePros) / (1 - (1 + rentePros) ** -måneder)
    print("\nAntall måneder:    ", måneder)
    return terminbeløp, rentePros, måneder


def lånOversikt(terminbeløp, rentePros, lånmen, måneder):
    restlån = lånmen
    i = 1
    renteArray = array.array('f', [])
    avdragArray = array.array('f', [])
    for betalinger in range(1, måneder + 1):
        rentebeløp = restlån * rentePros
        avdragbeløp = terminbeløp - rentebeløp
        restlån -= avdragbeløp
        renteArray.insert(betalinger - 1, rentebeløp)
        avdragArray.insert(betalinger - 1, avdragbeløp)

        if restlån < 1:
            restlån = 0
        headerFmt = ' {0:10}{1:10}{2:10}{3:10}{4:20s}'
        valueFmt  = '{0:5d}{1:12,.2f}{2:10,.2f}{3:12,.2f}{4:12,.2f}'
        print(headerFmt.format('Måned','Rente','Beløp','Avdrag','Restlån'))
        print(valueFmt.format(betalinger, rentebeløp, terminbeløp, avdragbeløp, restlån))
        if restlån == 0:
            print("\n\nGratulerer, lånet er nedbetalt")
        i += 1
    #print(renteArray)
    #print(avdragArray)
    plot(renteArray, avdragArray)


def plot(rentebeløp, avdragbeløp):
    xpoints = np.array(rentebeløp)
    ypoints = np.array(avdragbeløp)
    plt.plot(xpoints, ypoints, color='r')
    plt.title("Renter og avdrag på et annuitetslån")
    plt.xlabel("Renter")
    plt.ylabel("Avdrag")
    plt.show()


def main():
    lånStørrelse, rente, terminer = lånInfo()
    betalingsMengde, rentePros, måneder = beregneMånedligeBetalinger(lånStørrelse, rente, terminer)
    lånOversikt(betalingsMengde, rentePros, lånStørrelse, måneder)


main()

KeyboardInterrupt: Interrupted by user

Ved å fylle inn verdiene, 5 500 000 kr i lån, 2,14% mot 2,02% rente og 30 mot 27 år ser vi at terminbeløpene er lavere hos Sparebank 1, der de kun behøver å betale 16 965,20 kr i måneden, mens hos DNB er terminbeløpene på    18 752 kr i måneden.

Denne kalkulatoren er et fint verktøy for meg å ha i fremtiden, i et evig økende boligmarked vil det være godt å ha oversikt over hvor mye en blir nødt å betale i måneden for å betale ned et lån uansett størrelse, rente og terminer. 